In [1]:
#load aiida environment and connect to database
from aiida import load_profile

profile = load_profile()
print('profile name : ',profile.name)
print('profile database host name : ',profile.database_hostname)
print('default user : ',profile.default_user)

profile name :  default
profile database host name :  localhost
default user :  r.mozumder@fz-juelich.de


In [2]:
# load classes and functions
from aiida.orm import Code, Dict, load_node, StructureData
# from aiida.orm import
from aiida_kkr.tools import kkrparams
from aiida.engine import submit
from aiida_kkr.tools import kkrparams, plot_kkr
from aiida_kkr.workflows import kkr_flex_wc, kkr_imp_dos_wc, kkr_dos_wc

from aiida.orm import load_group, WorkChainNode
import matplotlib.pyplot as plt

In [3]:
# load classes and functions
from aiida.orm import CifData, Code, Dict, load_node, StructureData 
from aiida.orm import (Group, load_group, load_node, load_code, groups,
                      WorkChainNode)
from aiida.engine import submit
from aiida.common.exceptions import NotExistent
from aiida_kkr.workflows import kkr_imp_sub_wc, kkr_imp_dos, kkr_imp_wc, kkr_startpot_wc
import numpy as np
from aiida_kkr.calculations import KkrimpCalculation, VoronoiCalculation
import matplotlib.pyplot as plt
from aiida_kkr.tools.common_workfunctions import get_username

In [4]:
## Set the metadata options and code
metadata_option_1 = {'max_wallclock_seconds': 36000,'resources': 
               {'tot_num_mpiprocs': 48, 'num_machines': 1},
              'custom_scheduler_commands': 
              '#SBATCH --account=jara0191\n\nulimit -s unlimited; export OMP_STACKSIZE=2g',
              'withmpi': True
                    }

oscar_matadata = {'max_wallclock_seconds': 8*60*60,
                  'resources':{'tot_num_mpiprocs': 12, 
                  'num_machines': 4},
                  'custom_scheduler_commands': '#SBATCH -u mozumder -p oscar\n\nulimit -s unlimited; export OMP_STACKSIZE= 2g',
                  'withmpi':True
                 }
voro_code = Code.get_from_string('voro@claix18_init')
kkr_code = Code.get_from_string('kkr@claix18_init')
kkrimp_code = Code.get_from_string('kkrflex@claix18_init')
iffslurm_voro = Code.get_from_string('voro@iffslurm')
iffslurm_kkr= Code.get_from_string('kkr@iffslurm')
iffslurm_kkrimp= Code.get_from_string('kkrflex@iffslurm')

In [6]:
# Loading the converged kkr host calculation and the remote foder
Bi2Te3_conv_kkr = load_node('23b13d9d-6e02-46c7-b3ac-17033b205b19')#pk:245
remote_last_calc_folder = Bi2Te3_conv_kkr.outputs.remote_folder


In [18]:
# Previous DOS calculation for settings input parameter

delete_node= [104946,104952,]
node_59265= load_node(59265)
wf_parameter= node_59265.inputs.wf_parameters.get_dict()
wf_parameter= Dict(dict={'emax': 1.0, 'emin': -1.0, 'nepts': 96, "tempr": 800, "kmesh": [50,50,50]})

In [19]:
label= 'DOS Calc for Bi2Te3 host'

kkrdoswc = kkr_dos_wc.get_builder()

kkrdoswc.kkr = kkr_code
kkrdoswc.options = Dict(dict=metadata_option_1)
kkrdoswc.wf_parameters = wf_parameter
kkrdoswc.metadata.label = label
kkrdoswc.remote_data= remote_last_calc_folder
#host_dos= submit(kkrdoswc) # 104958 sucessfull

In [21]:
host_dos

<WorkChainNode: uuid: 59d5d1ae-5a91-415a-9510-04710dd8089f (pk: 104958) (aiida.workflows:kkr.dos)>